In [1]:
from model import * 
from preprocess import * 
from training_manager import *

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_min = 0.0
x_max = 1.0
y_min = 0.0
y_max = 0.4
ub = np.array([x_max, y_max])
lb = np.array([x_min, y_min])
data_loader = batch_maker(cylinder=[0.2,0.2,0.05],domain_ub=[x_max,y_max],domain_lb=[x_min,y_min],A=1,Centered=True)

In [3]:
#for the first phase
cyl_xy  = data_loader.cylinder2d(300)
cyl_uv = np.zeros((300+4, 2))

domain_square = data_loader.around_obstacle(1849)
total_domain = data_loader.design_batches(10,1849,1849)
total_domain.append(domain_square)
###############################################################################
'''
**Important Note:** The first loss function, referred to as the Primary Loss Function in the article, only includes a subset of boundary conditions. 
Therefore, to construct this loss function, only a subset of boundary conditions should be defined as `xy_bd` and `uv_bd`. 
'''
###############################################################################

wall_xy,wall_uv,_,_,inlet_xy,inlet_uv = data_loader.boundary(1000, 1000)

# wall_cycl and outlet pressure is exculded from xy_bd
xy_bd = np.concatenate([wall_xy,inlet_xy], axis=0)
xy_bd = torch.tensor(xy_bd, dtype=torch.float32).to(DEVICE)
uv_bd = np.concatenate([wall_uv,inlet_uv], axis=0)
uv_bd = torch.tensor(uv_bd, dtype=torch.float32).to(DEVICE)

tensor_total_domain = []
for i in total_domain:
  i = torch.tensor(i, dtype=torch.float32).to(DEVICE)
  tensor_total_domain.append(i)


In [4]:
EPOCHS = 10000
#variance reduction and preparing domain points using the inital weight state of different neural networks
new_model,result = pipe_line([40,40,40,40],tensor_total_domain,ub,lb,n=10,PDE='Navier_Stokes')
#create the model
model_pipe = PINN_Net('Navier_Stokes', [40,40,40,40],ub,lb,nn.Tanh()).to(DEVICE)
# load the initial state with reduced variance
model_pipe.load_state_dict(new_model)
# First phase of train
trainer_pipe = PINNManager(model_pipe.to(DEVICE),3e-4,'Navier_Stokes')
losse_bc,losses_pde,model_pipe_trained = trainer_pipe.adam_optimizer(EPOCHS,result,xy_bd,uv_bd)

 10%|███████▋                                                                    | 1019/10000 [00:07<01:02, 144.70it/s]

Epoch: 999, Loss: 0.008030088618397713, bc: 0.0031161224469542503, PDE: 0.004913965705782175


 20%|███████████████▍                                                            | 2027/10000 [00:14<00:56, 141.38it/s]

Epoch: 1999, Loss: 0.0005651228711940348, bc: 0.00021668057888746262, PDE: 0.0003484422923065722


 30%|███████████████████████                                                     | 3027/10000 [00:20<00:45, 153.22it/s]

...

Epoch: 7999, Loss: 2.6806188543559983e-05, bc: 8.74093166203238e-06, PDE: 1.8065256881527603e-05


 90%|████████████████████████████████████████████████████████████████████▌       | 9027/10000 [01:00<00:06, 151.41it/s]

Epoch: 8999, Loss: 2.070401023956947e-05, bc: 7.424153409374412e-06, PDE: 1.327985773968976e-05


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [01:06<00:00, 150.24it/s]

Epoch: 9999, Loss: 1.800814425223507e-05, bc: 6.653247510257643e-06, PDE: 1.1354895832482725e-05


- Replacing a proportion of random weights with `smart weights` (the weights created from the previous phase of training) 
in the output layer and the first few layers.

In [5]:
#Increasing complexity by adding random weights to smart weights
last_model = PINN_Net('Navier_Stokes', [40,40,40,40,40,40,40,40,40,40],ub,lb,nn.Tanh()).to(DEVICE)
last_model_updated_state = load_smart_weights(last_model,model_pipe_trained) 
last_model.load_state_dict(last_model_updated_state)


<All keys matched successfully>

In [6]:
# Second Phase of training with complete boundary conditions
LR= 1
Epochs = 400000
xy_col, xy_bnd, uv_bnd, xy_outlet = prepare_data(N_c=50000) #preparing domain points which PDE residual will be calculated on it
trainer_ = PINNManager(last_model.to(DEVICE),LR,'Navier_Stokes')
total_loss, model_trained_ = trainer_.lbfgs_optimizer(Epochs,xy_col,xy_bnd,uv_bnd,xy_outlet)

  0%|                                                                             | 6/400000 [00:00<2:13:57, 49.77it/s]

Epoch: 0, Loss: 2.1202948093414307


  0%|                                                                           | 107/400000 [00:02<3:18:38, 33.55it/s]

Epoch: 100, Loss: 0.04013194143772125


  0%|                                                                           | 207/400000 [00:05<3:21:34, 33.06it/s]

...

 10%|███████▎                                                                 | 40406/400000 [20:22<2:59:34, 33.38it/s]

Epoch: 40400, Loss: 0.00010077251499751583


 10%|███████▍                                                                 | 40506/400000 [20:25<3:01:41, 32.98it/s]

Epoch: 40500, Loss: 0.00010026506060967222


 10%|███████▍                                                                 | 40554/400000 [20:26<3:01:15, 33.05it/s]
